CNN for Sarcasm detection

In [11]:
import urllib.request
import zipfile
import os

# URL of the GloVe embeddings
glove_url = "http://nlp.stanford.edu/data/glove.6B.zip"
glove_zip = "glove.6B.zip"

# Download the file
print("Downloading GloVe embeddings...")
urllib.request.urlretrieve(glove_url, glove_zip)
print("Download complete!")

# Extract the zip file
print("Extracting files...")
with zipfile.ZipFile(glove_zip, "r") as zip_ref:
    zip_ref.extractall()
print("Extraction complete!")

# Remove the zip file (optional)
#os.remove(glove_zip)


Download complete!
Extracting files...
Extraction complete!


In [ ]:
import pandas as pd
import json
with open("Sarcasm_Headlines_Dataset.json", "r") as file:
    data = [json.loads(line) for line in file]
df = pd.DataFrame(data)
print(df.head())

                                        article_link  \
0  https://www.huffingtonpost.com/entry/versace-b...   
1  https://www.huffingtonpost.com/entry/roseanne-...   
2  https://local.theonion.com/mom-starting-to-fea...   
3  https://politics.theonion.com/boehner-just-wan...   
4  https://www.huffingtonpost.com/entry/jk-rowlin...   

                                            headline  is_sarcastic  
0  former versace store clerk sues over secret 'b...             0  
1  the 'roseanne' revival catches up to our thorn...             0  
2  mom starting to fear son's web series closest ...             1  
3  boehner just wants wife to listen, not come up...             1  
4  j.k. rowling wishes snape happy birthday in th...             0  


In [ ]:
text = df['headline'].values
labels = df['is_sarcastic'].values
# Tokenization
vocab_size = 10000  # Keep top 10K words
max_length = 30  # Maximum words per sentence
oov_token = "<OOV>"  # Handle unknown words
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(text)
# Convert to sequences and pad them
sequences = tokenizer.texts_to_sequences(text)
X = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")
# Convert labels to numpy array
y = labels
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42, test_size=0.2)
print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")

Training samples: 21367, Test samples: 5342


In [19]:
import pandas as pd
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Glove embeddings
embedding_index = {}
embedding_dim = 100  # Use 100D embeddings

with open("glove.6B.100d.txt", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coefs

# Create embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, index in tokenizer.word_index.items():
    if index < vocab_size:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

print("Embedding matrix shape:", embedding_matrix.shape)


Embedding matrix shape: (10000, 100)


In [ ]:
vocab_size = 10000   # Match tokenizer
embedding_dim = 100  # Match GloVe dimension
max_length = 100     # Sentence length
num_classes = 2      # Binary classification

# Define CNN model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim,
              weights=[embedding_matrix], trainable=False, input_length=max_length),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Build model explicitly
model.build(input_shape=(None, max_length))

#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Print summary
model.summary() # Compile the model

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ (None, 100, 100)            │       1,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_8 (Conv1D)                    │ (None, 96, 128)             │          64,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_8               │ (None, 128)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,072,514 (4.09 MB)

 Trainable params: 72,514 (283.26 KB)

 Non-trainable params: 1,000,000 (3.81 MB)

Epoch 1/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.6519 - loss: 0.6232 - val_accuracy: 0.8034 - val_loss: 0.4214
Epoch 2/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.8277 - loss: 0.3929 - val_accuracy: 0.8194 - val_loss: 0.3906
Epoch 3/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.8707 - loss: 0.3055 - val_accuracy: 0.8235 - val_loss: 0.3808
Epoch 4/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.9125 - loss: 0.2259 - val_accuracy: 0.7963 - val_loss: 0.4660
Epoch 5/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9359 - loss: 0.1683 - val_accuracy: 0.8272 - val_loss: 0.4366


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9524 - loss: 0.1202 - val_accuracy: 0.8235 - val_loss: 0.5358
Epoch 2/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9731 - loss: 0.0789 - val_accuracy: 0.8218 - val_loss: 0.5526
Epoch 3/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.9727 - loss: 0.0724 - val_accuracy: 0.8263 - val_loss: 0.6772
Epoch 4/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9796 - loss: 0.0568 - val_accuracy: 0.8231 - val_loss: 0.7637
Epoch 5/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9838 - loss: 0.0454 - val_accuracy: 0.7917 - val_loss: 0.9694


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
l, a = model.evaluate(X_train, y_train)
print(f"Train Accuracy: {a * 100:.2f}%")

167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8232 - loss: 0.4366
Test Accuracy: 82.72%
668/668 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9756 - loss: 0.0920
Train Accuracy: 97.57%


In [ ]:
from textblob import TextBlob

def sentiment_analysis(text):
    sentiment_score = TextBlob(text).sentiment.polarity
    if sentiment_score > 0:
        return "Positive Sarcasm"
    elif sentiment_score < 0:
        return "Negative Sarcasm"
    else:
        return "Neutral"
# Example
sample_text = "you are genius, obviously"
print(sentiment_analysis(sample_text))

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x799bbe8acfe0>
Traceback (most recent call last):
  File "/usr/lib/python3.11/_weakrefset.py", line 39, in _remove
    def _remove(item, selfref=ref(self)):

KeyboardInterrupt: 


Neutral


Trying transformers for sarcasm detection: Pretrained models from hugging face. Models used: RoBERTa and DistilBERT

In [ ]:
from transformers import pipeline
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
sentiment_task("Oh yeah, like Kitty's gonna care")

NameError: name 'model_path' is not defined

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
text = "Covid cases are increasing fast!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf

# Load the pre-trained tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)  # Binary classification for sarcasm (sarcastic / not sarcastic)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
# Assuming the dataset has columns 'text' and 'label'
X = text.tolist()  # List of texts
y = labels.tolist()  # Labels (0: non-sarcastic, 1: sarcastic)

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Tokenize the training and testing data
X_train_encoded = tokenizer(X_train, padding=True, truncation=True, max_length=128, return_tensors="tf")
X_test_encoded = tokenizer(X_test, padding=True, truncation=True, max_length=128, return_tensors="tf")

In [ ]:
# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_train_encoded['input_ids'], "attention_mask": X_train_encoded['attention_mask']},
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_test_encoded['input_ids'], "attention_mask": X_test_encoded['attention_mask']},
    y_test
))

# Batch the datasets for training
train_dataset = train_dataset.batch(16).shuffle(1000)
test_dataset = test_dataset.batch(16)

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                     │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11 (44.00 B)

 Trainable params: 11 (44.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
# !pip install transformers

   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.2 MB 8.6 MB/s eta 0:00:02
   -- ------------------------------------- 0.6/10.2 MB 7.1 MB/s eta 0:00:02
   --- ------------------------------------ 0.8/10.2 MB 5.9 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/10.2 MB 6.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.4/10.2 MB 5.9 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/10.2 MB 5.8 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/10.2 MB 5.6 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/10.2 MB 5.8 MB/s eta 0:00:02
   -------- ------------------------------- 2.2/10.2 MB 5.1 MB/s eta 0:00:02
   --------- ------------------------------ 2.5/10.2 MB 5.4 MB/s eta 0:00:02
   ---------- ----------------------------- 2.7/10.2 MB 5.3 MB/s eta 0:00:02
   ----------- ---------------------------- 3.0/10.2 MB 5.3 MB/s eta 0:00:02
   ---

In [2]:
import pandas as pd

# Load dataset (adjust the path based on where you stored the file)
df = pd.read_json("./Sarcasm_Headlines_Dataset.json", lines=True)
# View the first few rows
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [4]:
df["is_sarcastic"].value_counts(normalize=True)


is_sarcastic
0    0.561047
1    0.438953
Name: proportion, dtype: float64

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilbert-finetuned-sarcasm-classification")

# Tokenize the headlines
def tokenize_data(examples):
    return tokenizer(examples["headline"], truncation=True, padding="max_length", max_length=128)

df["label"] = df["is_sarcastic"]  # Rename column for clarity

# Convert to Hugging Face dataset format
from datasets import Dataset

dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize_data, batched=True)
dataset = dataset.train_test_split(test_size=0.2)

train_dataset = dataset["train"]
test_dataset = dataset["test"]

Map:   0%|          | 0/26709 [00:00<?, ? examples/s]

In [7]:
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import TrainingArguments, Trainer
import torch

# Define training arguments
training_args = TrainingArguments(
    output_dir="./sarcasm_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

C:\Users\bavna\anaconda3\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\bavna\AppData\Local\Temp\ipykernel_22484\3888565699.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

C:\Users\bavna\AppData\Local\Temp\ipykernel_22484\3689599755.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.236700,0.179501
2,0.106400,0.222543
3,0.029000,0.323299


TrainOutput(global_step=4008, training_loss=0.14245249719141487, metrics={'train_runtime': 26497.8884, 'train_samples_per_second': 2.419, 'train_steps_per_second': 0.151, 'total_flos': 2122823180312064.0, 'train_loss': 0.14245249719141487, 'epoch': 3.0})

In [13]:
# Evaluate the model on the test dataset
eval_results = trainer.evaluate(test_dataset)

# Print evaluation results
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 0.1795007437467575, 'eval_runtime': 1194.3928, 'eval_samples_per_second': 4.473, 'eval_steps_per_second': 0.28, 'epoch': 3.0}


In [15]:
import numpy as np
from sklearn.metrics import accuracy_score

# Get model predictions on the training dataset
train_predictions = trainer.predict(train_dataset)

# Convert logits to class labels (0 = Not Sarcastic, 1 = Sarcastic)
train_pred_labels = np.argmax(train_predictions.predictions, axis=1)

# Compute accuracy
train_accuracy = accuracy_score(train_dataset["label"], train_pred_labels)

print(f"Training Accuracy: {train_accuracy:.4f}")


Training Accuracy: 0.9718


In [21]:
# Get model predictions on the training dataset
test_predictions = trainer.predict(test_dataset)

# Convert logits to class labels (0 = Not Sarcastic, 1 = Sarcastic)
test_pred_labels = np.argmax(test_predictions.predictions, axis=1)

# Compute accuracy
test_accuracy = accuracy_score(test_dataset["label"], test_pred_labels)

print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.9274


In [19]:
# Save the trained model and tokenizer
model.save_pretrained("./sarcasm_model")
tokenizer.save_pretrained("./sarcasm_model")

('./sarcasm_model\\tokenizer_config.json',
 './sarcasm_model\\special_tokens_map.json',
 './sarcasm_model\\vocab.txt',
 './sarcasm_model\\added_tokens.json',
 './sarcasm_model\\tokenizer.json')

Sentiment Analysis using roBERTa

In [3]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import numpy as np
from scipy.special import softmax

model_name = "cardiffnlp/twitter-roberta-base-sentiment"  # example
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

C:\Users\bavna\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bavna\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/501M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [9]:
X_test = [
    "I love being ignored. So fun.",
    "You're the best friend ever!",
    "Go cry about it",
]

y_test = [1, 0, 1]  # assuming 1 = cyberbullying/sarcasm, 0 = not
y_pred = []

for text in X_test:
    inputs = tokenizer(
    X_test,               # list of texts
    return_tensors="tf",
    padding=True,
    truncation=True,
    max_length=128 )
    outputs = model(inputs)
    logits = outputs.logits.numpy()[0]
    probs = softmax(logits)
    pred_label = np.argmax(probs)
    y_pred.append(pred_label)

In [13]:
from sklearn.metrics import classification_report

# Update target_names to match the 3 sentiment labels
target_names = ['negative', 'neutral', 'positive']

print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00       1.0
     neutral       0.00      0.00      0.00       2.0
    positive       0.00      0.00      0.00       0.0

    accuracy                           0.00       3.0
   macro avg       0.00      0.00      0.00       3.0
weighted avg       0.00      0.00      0.00       3.0



C:\Users\bavna\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bavna\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bavna\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bavna\anaconda3\Lib\site-packages\sklea

In [1]:
from datasets import load_dataset

# Load sentiment subset of TweetEval
dataset = load_dataset("tweet_eval", "sentiment")

In [7]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from scipy.special import softmax
import numpy as np

model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [14]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from datasets import load_dataset
from scipy.special import softmax
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Load model & tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Load dataset
dataset = load_dataset("tweet_eval", "sentiment")
X_test = dataset["test"]["text"]
y_test = dataset["test"]["label"]

# Batch tokenize
batch_size = 32
y_pred = []

for i in range(0, len(X_test), batch_size):
    batch_texts = X_test[i:i+batch_size]
    inputs = tokenizer(batch_texts, return_tensors="tf", padding=True, truncation=True, max_length=128)
    
    # Batch prediction
    outputs = model(inputs)
    batch_logits = outputs.logits.numpy()
    batch_probs = softmax(batch_logits, axis=1)
    batch_preds = np.argmax(batch_probs, axis=1)

    y_pred.extend(batch_preds)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["negative", "neutral", "positive"]))

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Accuracy: 0.7244382937154021
              precision    recall  f1-score   support

    negative       0.71      0.78      0.74      3972
     neutral       0.74      0.69      0.71      5937
    positive       0.72      0.71      0.71      2375

    accuracy                           0.72     12284
   macro avg       0.72      0.73      0.72     12284
weighted avg       0.73      0.72      0.72     12284



Sentiment - Finetune

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate

# ✅ Step 3: Load TweetEval Sentiment dataset
dataset = load_dataset("tweet_eval", "sentiment")
# ✅ Step 4: Load tokenizer and model
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

In [7]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

tokenized_ds = dataset.map(tokenize, batched=True)
tokenized_ds = tokenized_ds.remove_columns(["text"])
tokenized_ds.set_format("torch")

Map:   0%|          | 0/12284 [00:00<?, ? examples/s]

In [9]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [11]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

C:\Users\bavna\anaconda3\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [3]:
from datasets import load_dataset
sst = load_dataset("glue", "sst2")
print(sst)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

C:\Users\bavna\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bavna\.cache\huggingface\hub\datasets--glue. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from scipy.special import softmax
import pandas as pd
from tqdm import tqdm

# Load pretrained sarcasm model
model_name = "jkhan447/sarcasm-detection-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
df = pd.read_json("Sarcasm_Headlines_Dataset.json", lines=True)
df = df[["headline", "is_sarcastic"]].dropna()
df = df[:5000]
all_preds = []
all_labels = df["is_sarcastic"].tolist()

for text in tqdm(df["headline"]):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = softmax(logits.cpu().numpy(), axis=1)
        pred = probs.argmax(axis=1).item()
    all_preds.append(pred)
    

from sklearn.metrics import classification_report

print(classification_report(all_labels, all_preds, target_names=["Not Sarcastic", "Sarcastic"]))


100%|██████████| 5000/5000 [03:10<00:00, 26.29it/s]

               precision    recall  f1-score   support

Not Sarcastic       0.54      0.40      0.46      2813
    Sarcastic       0.42      0.55      0.48      2187

     accuracy                           0.47      5000
    macro avg       0.48      0.48      0.47      5000
 weighted avg       0.48      0.47      0.47      5000



In [33]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from scipy.special import softmax
import torch
from tqdm import tqdm

# ✅ Step 1: Load the sarcasm dataset (TSV format)
df = pd.read_csv("train-balanced-sarc.csv", sep="\t", header=None, on_bad_lines="skip")
df.columns = ["label", "author", "subreddit", "score", "parent", "comment_id", "id", "created_utc", "context", "text"]


df = df.dropna()
df = df[:500]

print(df['label'])
# ✅ Step 3: Load pretrained sarcasm classifier (TensorFlow weights → PyTorch)
model_name = "mrm8488/distilbert-finetuned-sarcasm-classification"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, from_tf=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# ✅ Step 4: Inference loop
all_preds, all_labels = [], []

for _, row in tqdm(df.iterrows(), total=len(df)):
    text = str(row["text"])  # ✅ Force it to be string
    label = int(row["label"])

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = softmax(logits.cpu().numpy(), axis=1)
        pred = probs.argmax(axis=1).item()
    text = str(row["text"])

    print(f"Text: {text}")
    print(f"Sarcastic probability: {probs[0][1]:.4f}")


    all_preds.append(pred)
    all_labels.append(label)


# ✅ Step 5: Evaluation
print(classification_report(all_labels, all_preds, target_names=["Not Sarcastic", "Sarcastic"]))


0      0
1      0
2      0
3      0
4      0
      ..
495    0
496    0
497    1
498    0
499    0
Name: label, Length: 500, dtype: int64


All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.
  2%|▏         | 10/500 [00:00<00:10, 47.27it/s]

Text: Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well.
Sarcastic probability: 0.0005
Text: The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.
Sarcastic probability: 0.0002
Text: They're favored to win.
Sarcastic probability: 0.0002
Text: deadass don't kill my buzz
Sarcastic probability: 0.9963
Text: Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT
Sarcastic probability: 0.0120
Text: do you find ariana grande sexy ?
Sarcastic probability: 0.0002
Text: What's your weird or unsettling Trick or Treat story?
Sarcastic probability: 0.0002
Text: Probably Sephiroth. I refuse to taint his greatness with *my* weak builds. He should equip *ONLY* the best TMs.
Sarcastic probability: 0.0005
Text: What to upgrade? I have $500 to spend (mainly because it's my birthday on the 31st) and I'm not really sure what to upgrade. I don't have to spend all $50

  3%|▎         | 15/500 [00:00<00:10, 47.30it/s]

Text: I bet if that money was poured into college debt or health debt relief, 81% of Americans would have been for it instead.
Sarcastic probability: 0.0003
Text: James Shields Will Not Exercise Opt-Out Clause
Sarcastic probability: 0.1049
Text: There's no time to worry about leaks when they gays are already out to get you.
Sarcastic probability: 0.0002
Text: Team Specific Threads No matter how many articles you read or how many hours of Red Zone you watch nothing quite makes up for watching every play of a game. Would the members of this community/the mods be open to creating a sticky post that has a team individual thread for every team in the NFL where team specific conversation can take place and people that watch individual games can comment? We may even be able to get some guys from team specific subreddits to stop by and give us some new insight.
Sarcastic probability: 0.0817
Text: Ill give you a hint. They commented in this thread
Sarcastic probability: 0.3170
Text: Star Wars, 

...

Text: Burried in a grave filled with chinese money and hunnies, next to Dade.
Sarcastic probability: 0.0003
Text: When No Game No Life S2 comes out
Sarcastic probability: 0.0037
Text: To be fair, anyone you have to nag for a rent payment should find somewhere else to live.
Sarcastic probability: 0.0003
Text: I have tested bringing this third spell into the mix, however I would personally not recommend it. There is specific timing required for the spin flax spell, and adding another strange timing factor into it does increase xp/hr, but only by about ~6% and decreases the already lowish profit you make from spin flax + tan leather method.
Sarcastic probability: 0.0004
Text: glad i left that shit hole
Sarcastic probability: 0.9986
Text: Did i say only?
Sarcastic probability: 0.0036
Text: What was the other one?
Sarcastic probability: 0.0159
Text: I hope they scrap Murphy after this season though. He serves no real purpose after Escobar. Would have been better if Carillo was around as one

 98%|█████████▊| 492/500 [00:11<00:00, 43.26it/s]

Text: yeah that guy won't recover for a while...
Sarcastic probability: 0.9996
Text: I saw it described once that "boys are for fun, women are for babies." You can also s/boys/goats/ and it's probably still true.
Sarcastic probability: 0.0047
Text: What are you guys talking a boat?
Sarcastic probability: 0.0044
Text: being forced to watch a hitlery speech is cruel and unusual punishment. time to lawyer up.
Sarcastic probability: 0.0002
Text: He doesn't think you can roast him.
Sarcastic probability: 0.0007
Text: It's also a neighborhood in San Diego, and a beach in San Francisco.
Sarcastic probability: 0.0002
Text: yep, yak9ut also has 3 loadouts for guns. 2x20 and 23, 2x20 and 37, 2x20 and 45. or was it the yak9p?
Sarcastic probability: 0.0009
Text: To be honest, I don't know. But I do know the Director of the FBI should know how.
Sarcastic probability: 0.0002
Text: For 3 Years I've Been Playing On A Laptop - What Have I Missed Today was a good day. I've been playing solely PC for 3 y

100%|██████████| 500/500 [00:11<00:00, 43.38it/s]

Text: Who got what from the steam sale? I'll start: * Doom * Witcher 3 * Xcom 2 * Shadow of Mordor * Mortal Kombat x * Portal 2 * Divinity Org Sin * Shadowrun Returns * FTL I recon I'm set for a while. Trouble is I've started with FTL and I'm hooked!
Sarcastic probability: 0.0003
Text: Can we have a "Sorry" in the context menu or a hotkey? I often do mistakes and I want to apologise to my team for when I do something dumb. Please Blizzard, this is my one request!
Sarcastic probability: 0.0003
Text: It doesn't bother me actually. Its a non-issue for consumers, but still weird that they chose to market it. Wonder why.
Sarcastic probability: 0.0002
Text: The shuttle orbiter is 37 meters long, so the spaceX Mars vehicle is about three times as long and many more times as massive
Sarcastic probability: 0.0002
Text: System requirements always tend to be estimates.
Sarcastic probability: 0.0007
               precision    recall  f1-score   support

Not Sarcastic       0.95      0.92      0.9

In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from scipy.special import softmax
import torch
from tqdm import tqdm

# ✅ Step 1: Load the sarcasm dataset
df = pd.read_csv("train-balanced-sarc.csv", sep="\t", header=None, on_bad_lines="skip")
df.columns = ["label", "author", "subreddit", "score", "parent", "comment_id", "id", "created_utc", "context", "text"]
df = df.dropna()
df = df[:500]  # Use a sample subset for quick testing

df["label"] = df["label"].astype(int)
df["text"] = df["text"].astype(str)

# ✅ Step 2: Load model and tokenizer
model_name = "helinivan/english-sarcasm-detector"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# ✅ Step 3: Inference loop
all_preds, all_labels = [], []

for _, row in tqdm(df.iterrows(), total=len(df)):
    text = row["text"]
    label = row["label"]

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits
        probs = softmax(logits.cpu().numpy(), axis=1)
        pred = probs.argmax(axis=1).item()

    # Debug print
    print(f"Text: {text}")
    print(f"Model output: {probs}")
    print(f"Predicted label: {'Sarcastic' if pred == 1 else 'Not Sarcastic'}\n")

    all_preds.append(pred)
    all_labels.append(label)

# ✅ Step 4: Evaluation
print("\nFinal Classification Report:")
print(classification_report(all_labels, all_preds, target_names=["Not Sarcastic", "Sarcastic"]))


  1%|          | 5/500 [00:00<00:23, 21.35it/s]

Text: Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well.
Model output: [[0.97111964 0.02888036]]
Predicted label: Not Sarcastic

Text: The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.
Model output: [[0.98927873 0.01072126]]
Predicted label: Not Sarcastic

Text: They're favored to win.
Model output: [[0.9909279  0.00907205]]
Predicted label: Not Sarcastic

Text: deadass don't kill my buzz
Model output: [[0.44531098 0.55468905]]
Predicted label: Sarcastic

Text: Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT
Model output: [[0.6506572  0.34934285]]
Predicted label: Not Sarcastic



  2%|▏         | 11/500 [00:00<00:20, 23.61it/s]

Text: do you find ariana grande sexy ?
Model output: [[0.9942726  0.00572748]]
Predicted label: Not Sarcastic

Text: What's your weird or unsettling Trick or Treat story?
Model output: [[0.99047476 0.0095253 ]]
Predicted label: Not Sarcastic

Text: Probably Sephiroth. I refuse to taint his greatness with *my* weak builds. He should equip *ONLY* the best TMs.
Model output: [[0.87580544 0.12419463]]
Predicted label: Not Sarcastic

Text: What to upgrade? I have $500 to spend (mainly because it's my birthday on the 31st) and I'm not really sure what to upgrade. I don't have to spend all $500, I could spend as little or all of it if I want. Specs are: CPU: i5-6600K Cooler: CM 212 EVO RAM: 8GB Corsair Vengence LPX 2400MHz GPU: EVGA GTX 1070 SC Case: Corsair Spec Alpha PSU: EVGA P2 650w Storage: One 480 GB Corsair Force LE Monitor: BenQ XL2411Z Keyboard: Razer Blackwidow Tournament Edition What I'm thinking of right now is: -Get a 1TB WD Blue -Buy another 8GB of RAM -Replace my Razer Blackwid

  3%|▎         | 17/500 [00:00<00:19, 24.23it/s]

...

Text: I just wish they added it as a part of FNM rather than a separate day. I have enough trouble getting out for one night of magic a week. Also, what times do your guys' stores do the event? Because my store thought it was a decent idea at 2pm...
Model output: [[0.9867215  0.01327845]]
Predicted label: Not Sarcastic

Text: yeah that guy won't recover for a while...
Model output: [[0.79188854 0.20811154]]
Predicted label: Not Sarcastic

Text: I saw it described once that "boys are for fun, women are for babies." You can also s/boys/goats/ and it's probably still true.
Model output: [[0.9909967  0.00900323]]
Predicted label: Not Sarcastic

Text: What are you guys talking a boat?
Model output: [[0.90583706 0.09416293]]
Predicted label: Not Sarcastic

Text: being forced to watch a hitlery speech is cruel and unusual punishment. time to lawyer up.
Model output: [[0.9957766  0.00422345]]
Predicted label: Not Sarcastic



 99%|█████████▊| 493/500 [00:20<00:00, 20.22it/s]

Text: He doesn't think you can roast him.
Model output: [[0.9488437  0.05115629]]
Predicted label: Not Sarcastic

Text: It's also a neighborhood in San Diego, and a beach in San Francisco.
Model output: [[0.99429166 0.00570837]]
Predicted label: Not Sarcastic

Text: yep, yak9ut also has 3 loadouts for guns. 2x20 and 23, 2x20 and 37, 2x20 and 45. or was it the yak9p?
Model output: [[0.9354101  0.06458993]]
Predicted label: Not Sarcastic

Text: To be honest, I don't know. But I do know the Director of the FBI should know how.
Model output: [[0.99103    0.00897005]]
Predicted label: Not Sarcastic

Text: For 3 Years I've Been Playing On A Laptop - What Have I Missed Today was a good day. I've been playing solely PC for 3 years, running integrated graphics and an I3. It overheats, runs CS at less than 60 frames. Let's just say, I've missed a lot when it comes to gaming. Now I've Ascended - a beautiful machine with an I7 and 1080 - what have I missed? What games have beautiful graphics? What

100%|██████████| 500/500 [00:21<00:00, 23.47it/s]

Text: Who got what from the steam sale? I'll start: * Doom * Witcher 3 * Xcom 2 * Shadow of Mordor * Mortal Kombat x * Portal 2 * Divinity Org Sin * Shadowrun Returns * FTL I recon I'm set for a while. Trouble is I've started with FTL and I'm hooked!
Model output: [[0.99264866 0.00735129]]
Predicted label: Not Sarcastic

Text: Can we have a "Sorry" in the context menu or a hotkey? I often do mistakes and I want to apologise to my team for when I do something dumb. Please Blizzard, this is my one request!
Model output: [[0.9959121  0.00408798]]
Predicted label: Not Sarcastic

Text: It doesn't bother me actually. Its a non-issue for consumers, but still weird that they chose to market it. Wonder why.
Model output: [[0.97196734 0.02803269]]
Predicted label: Not Sarcastic

Text: The shuttle orbiter is 37 meters long, so the spaceX Mars vehicle is about three times as long and many more times as massive
Model output: [[0.9931613  0.00683868]]
Predicted label: Not Sarcastic

Text: System req

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from scipy.special import softmax
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report

model_name = "helinivan/english-sarcasm-detector"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

df = pd.read_json("Sarcasm_Headlines_Dataset.json", lines=True)
df = df[["headline", "is_sarcastic"]].dropna()
df = df[:5000]

all_preds = []
all_labels = df["is_sarcastic"].tolist()

for text in tqdm(df["headline"]):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = softmax(logits.cpu().numpy(), axis=1)
        pred = probs.argmax(axis=1).item()
    all_preds.append(pred)

print(classification_report(all_labels, all_preds, target_names=["Not Sarcastic", "Sarcastic"]))

100%|██████████| 5000/5000 [07:24<00:00, 11.25it/s]   


               precision    recall  f1-score   support

Not Sarcastic       0.93      0.98      0.95      2813
    Sarcastic       0.97      0.91      0.94      2187

     accuracy                           0.95      5000
    macro avg       0.95      0.94      0.94      5000
 weighted avg       0.95      0.95      0.95      5000



In [20]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from scipy.special import softmax
import torch
from tqdm import tqdm

# ✅ Load the new dataset
df = pd.read_csv("sarcasm_tweets.csv")  # Replace with actual filename
df = df.dropna(subset=["Tweet", "Sarcasm (yes/no)"])

# ✅ Convert labels to 0/1
label_map = {"no": 0, "yes": 1}
df["label"] = df["Sarcasm (yes/no)"].map(label_map)
print(df["label"])
df["text"] = df["Tweet"].astype(str)

# ✅ Load model
model_name = "helinivan/english-sarcasm-detector"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# ✅ Inference
all_preds = []
all_labels = df["label"].tolist()

for text in tqdm(df["text"]):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = softmax(logits.cpu().numpy(), axis=1)[0]  # (2,) shape

    not_sarcastic_prob = float(probs[0])
    sarcastic_prob = float(probs[1])

    print(f"Text: {text}")
    print(f"→ Prob (Not Sarcastic): {not_sarcastic_prob:.4f} | Prob (Sarcastic): {sarcastic_prob:.4f}")

    
    print(f"Text: {text}")
    print(f"→ Prob (Not Sarcastic): {probs[0].item():.4f} | Prob (Sarcastic): {probs[1].item():.4f}")
    all_preds.append(pred)

# ✅ Evaluation
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Not Sarcastic", "Sarcastic"]))


0       0
1       0
2       1
3       1
4       1
       ..
4995    1
4996    0
4997    0
4998    0
4999    0
Name: label, Length: 5000, dtype: int64


  0%|          | 5/5000 [00:00<03:55, 21.20it/s]

Text: I genuinely love musicians.
→ Prob (Not Sarcastic): 0.9891 | Prob (Sarcastic): 0.0109
Text: I genuinely love musicians.
→ Prob (Not Sarcastic): 0.9891 | Prob (Sarcastic): 0.0109
Text: I find vegetarians to be wonderful.
→ Prob (Not Sarcastic): 0.9917 | Prob (Sarcastic): 0.0083
Text: I find vegetarians to be wonderful.
→ Prob (Not Sarcastic): 0.9917 | Prob (Sarcastic): 0.0083
Text: Because artists really make everything better.
→ Prob (Not Sarcastic): 0.9945 | Prob (Sarcastic): 0.0055
Text: Because artists really make everything better.
→ Prob (Not Sarcastic): 0.9945 | Prob (Sarcastic): 0.0055
Text: Can't wait for more of the game.
→ Prob (Not Sarcastic): 0.9589 | Prob (Sarcastic): 0.0411
Text: Can't wait for more of the game.
→ Prob (Not Sarcastic): 0.9589 | Prob (Sarcastic): 0.0411
Text: Oh great, another artists. Just what I needed.
→ Prob (Not Sarcastic): 0.9176 | Prob (Sarcastic): 0.0824
Text: Oh great, another artists. Just what I needed.
→ Prob (Not Sarcastic): 0.9176 | Pro

  0%|          | 8/5000 [00:00<03:43, 22.35it/s]

Text: this movie are so wonderful, said no one ever.
→ Prob (Not Sarcastic): 0.9760 | Prob (Sarcastic): 0.0240
Text: this movie are so wonderful, said no one ever.
→ Prob (Not Sarcastic): 0.9760 | Prob (Sarcastic): 0.0240
Text: Oh great, another my new phone. Just what I needed.
→ Prob (Not Sarcastic): 0.8476 | Prob (Sarcastic): 0.1524
Text: Oh great, another my new phone. Just what I needed.
→ Prob (Not Sarcastic): 0.8476 | Prob (Sarcastic): 0.1524
Text: Nothing better than a the meal moment.
→ Prob (Not Sarcastic): 0.9620 | Prob (Sarcastic): 0.0380
Text: Nothing better than a the meal moment.
→ Prob (Not Sarcastic): 0.9620 | Prob (Sarcastic): 0.0380
Text: Oh great, another writers. Just what I needed.
→ Prob (Not Sarcastic): 0.9185 | Prob (Sarcastic): 0.0815
Text: Oh great, another writers. Just what I needed.
→ Prob (Not Sarcastic): 0.9185 | Prob (Sarcastic): 0.0815
Text: I am always excited for the workshop.
→ Prob (Not Sarcastic): 0.9922 | Prob (Sarcastic): 0.0078
Text: I am alway

...

 13%|█▎        | 626/5000 [00:26<03:36, 20.16it/s]

Text: I just love when vegetarians happens.
→ Prob (Not Sarcastic): 0.9901 | Prob (Sarcastic): 0.0099
Text: I just love when vegetarians happens.
→ Prob (Not Sarcastic): 0.9901 | Prob (Sarcastic): 0.0099
Text: the game are so wonderful, said no one ever.
→ Prob (Not Sarcastic): 0.9833 | Prob (Sarcastic): 0.0167
Text: the game are so wonderful, said no one ever.
→ Prob (Not Sarcastic): 0.9833 | Prob (Sarcastic): 0.0167
Text: I find this book to be wonderful.
→ Prob (Not Sarcastic): 0.9931 | Prob (Sarcastic): 0.0069
Text: I find this book to be wonderful.
→ Prob (Not Sarcastic): 0.9931 | Prob (Sarcastic): 0.0069
Text: Nothing better than a the meal moment.
→ Prob (Not Sarcastic): 0.9620 | Prob (Sarcastic): 0.0380
Text: Nothing better than a the meal moment.
→ Prob (Not Sarcastic): 0.9620 | Prob (Sarcastic): 0.0380
Text: Nothing better than a scientists moment.
→ Prob (Not Sarcastic): 0.9718 | Prob (Sarcastic): 0.0282
Text: Nothing better than a scientists moment.
→ Prob (Not Sarcastic): 

 13%|█▎        | 630/5000 [00:27<03:07, 23.28it/s]

Text: Oh great, another this book. Just what I needed.
→ Prob (Not Sarcastic): 0.9500 | Prob (Sarcastic): 0.0500
Text: Oh great, another this book. Just what I needed.
→ Prob (Not Sarcastic): 0.9500 | Prob (Sarcastic): 0.0500
Text: I genuinely love this book.
→ Prob (Not Sarcastic): 0.9902 | Prob (Sarcastic): 0.0098
Text: I genuinely love this book.
→ Prob (Not Sarcastic): 0.9902 | Prob (Sarcastic): 0.0098


KeyboardInterrupt: 

In [12]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm
from sklearn.metrics import classification_report
from scipy.special import softmax

# ✅ Load the sarcasm dataset
dataset = load_dataset("nikesh66/Sarcasm-dataset")
df = dataset["train"].to_pandas()
df = df.dropna()

# ✅ Load model
model_name = "helinivan/english-sarcasm-detector"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# ✅ Inference
all_preds, all_labels = [], df["label"].tolist()

for text in tqdm(df["text"]):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = softmax(logits.cpu().numpy(), axis=1)
        pred = probs.argmax(axis=1).item()
    all_preds.append(pred)

# ✅ Evaluation
print(classification_report(all_labels, all_preds, target_names=["Not Sarcastic", "Sarcastic"]))


ConnectionError: Couldn't reach 'nikesh66/Sarcasm-dataset' on the Hub (LocalEntryNotFoundError)

In [5]:
print(classification_report(all_labels, all_preds, target_names=["Not Sarcastic", "Sarcastic"]))


               precision    recall  f1-score   support

Not Sarcastic       0.96      0.95      0.95       474
    Sarcastic       0.16      0.19      0.18        26

     accuracy                           0.91       500
    macro avg       0.56      0.57      0.56       500
 weighted avg       0.91      0.91      0.91       500



In [6]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from scipy.special import softmax
import torch
from tqdm import tqdm

# ✅ Step 1: Load the sarcasm dataset
df = pd.read_csv("train-balanced-sarc.csv", sep="\t", header=None, on_bad_lines="skip")
df.columns = ["label", "author", "subreddit", "score", "parent", "comment_id", "id", "created_utc", "context", "text"]
df = df.dropna()
df = df[:500]  # Use a sample for quick evaluation

df["label"] = df["label"].astype(int)
df["text"] = df["text"].astype(str)

# ✅ Step 2: Load model and tokenizer
model_name = "jkhan447/sarcasm-detection-Bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# ✅ Step 3: Inference loop
all_preds, all_labels = [], []

for _, row in tqdm(df.iterrows(), total=len(df)):
    text = row["text"]
    label = row["label"]

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits
        probs = softmax(logits.cpu().numpy(), axis=1)
        pred = probs.argmax(axis=1).item()

    # Debug print
    print(f"Text: {text}")
    print(f"Model output: {probs}")
    print(f"Predicted label: {'Sarcastic' if pred == 1 else 'Not Sarcastic'}\n")

    all_preds.append(pred)
    all_labels.append(label)

# ✅ Step 4: Evaluation
print("\nFinal Classification Report:")
print(classification_report(all_labels, all_preds, target_names=["Not Sarcastic", "Sarcastic"]))


  1%|          | 5/500 [00:00<00:21, 22.86it/s]

Text: Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well.
Model output: [[9.9999726e-01 2.7472411e-06]]
Predicted label: Not Sarcastic

Text: The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.
Model output: [[9.99986529e-01 1.34658585e-05]]
Predicted label: Not Sarcastic

Text: They're favored to win.
Model output: [[9.9997938e-01 2.0600482e-05]]
Predicted label: Not Sarcastic

Text: deadass don't kill my buzz
Model output: [[9.999968e-01 3.186717e-06]]
Predicted label: Not Sarcastic

Text: Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT
Model output: [[0.5418085  0.45819154]]
Predicted label: Not Sarcastic

Text: do you find ariana grande sexy ?
Model output: [[9.9999714e-01 2.8376687e-06]]
Predicted label: Not Sarcastic



  2%|▏         | 11/500 [00:00<00:19, 24.46it/s]

Text: What's your weird or unsettling Trick or Treat story?
Model output: [[9.9999702e-01 2.9691128e-06]]
Predicted label: Not Sarcastic

Text: Probably Sephiroth. I refuse to taint his greatness with *my* weak builds. He should equip *ONLY* the best TMs.
Model output: [[0.00313464 0.99686533]]
Predicted label: Sarcastic

Text: What to upgrade? I have $500 to spend (mainly because it's my birthday on the 31st) and I'm not really sure what to upgrade. I don't have to spend all $500, I could spend as little or all of it if I want. Specs are: CPU: i5-6600K Cooler: CM 212 EVO RAM: 8GB Corsair Vengence LPX 2400MHz GPU: EVGA GTX 1070 SC Case: Corsair Spec Alpha PSU: EVGA P2 650w Storage: One 480 GB Corsair Force LE Monitor: BenQ XL2411Z Keyboard: Razer Blackwidow Tournament Edition What I'm thinking of right now is: -Get a 1TB WD Blue -Buy another 8GB of RAM -Replace my Razer Blackwidow with a keyboard with actual Cherry MX switches -Replace my case (Doesn't really offer features for good ca

  3%|▎         | 17/500 [00:00<00:19, 24.53it/s]

...

Text: yeah that guy won't recover for a while...
Model output: [[9.9998653e-01 1.3431715e-05]]
Predicted label: Not Sarcastic

Text: I saw it described once that "boys are for fun, women are for babies." You can also s/boys/goats/ and it's probably still true.
Model output: [[9.999974e-01 2.662065e-06]]
Predicted label: Not Sarcastic

Text: What are you guys talking a boat?
Model output: [[9.9993026e-01 6.9702000e-05]]
Predicted label: Not Sarcastic

Text: being forced to watch a hitlery speech is cruel and unusual punishment. time to lawyer up.
Model output: [[0.75133824 0.24866179]]
Predicted label: Not Sarcastic

Text: He doesn't think you can roast him.
Model output: [[9.9996579e-01 3.4188906e-05]]
Predicted label: Not Sarcastic



 99%|█████████▉| 497/500 [00:20<00:00, 24.79it/s]

Text: It's also a neighborhood in San Diego, and a beach in San Francisco.
Model output: [[9.9999607e-01 3.9154520e-06]]
Predicted label: Not Sarcastic

Text: yep, yak9ut also has 3 loadouts for guns. 2x20 and 23, 2x20 and 37, 2x20 and 45. or was it the yak9p?
Model output: [[4.7277408e-05 9.9995267e-01]]
Predicted label: Sarcastic

Text: To be honest, I don't know. But I do know the Director of the FBI should know how.
Model output: [[5.4177764e-04 9.9945825e-01]]
Predicted label: Sarcastic

Text: For 3 Years I've Been Playing On A Laptop - What Have I Missed Today was a good day. I've been playing solely PC for 3 years, running integrated graphics and an I3. It overheats, runs CS at less than 60 frames. Let's just say, I've missed a lot when it comes to gaming. Now I've Ascended - a beautiful machine with an I7 and 1080 - what have I missed? What games have beautiful graphics? What story have I not played because my computer couldn't handle it? **Psst - I'm asking for you to recommen

100%|██████████| 500/500 [00:21<00:00, 23.73it/s]

Text: It doesn't bother me actually. Its a non-issue for consumers, but still weird that they chose to market it. Wonder why.
Model output: [[9.9999726e-01 2.7868655e-06]]
Predicted label: Not Sarcastic

Text: The shuttle orbiter is 37 meters long, so the spaceX Mars vehicle is about three times as long and many more times as massive
Model output: [[9.9998832e-01 1.1708279e-05]]
Predicted label: Not Sarcastic

Text: System requirements always tend to be estimates.
Model output: [[9.9999726e-01 2.7946103e-06]]
Predicted label: Not Sarcastic


Final Classification Report:
               precision    recall  f1-score   support

Not Sarcastic       0.96      0.73      0.83       474
    Sarcastic       0.09      0.46      0.14        26

     accuracy                           0.71       500
    macro avg       0.52      0.59      0.49       500
 weighted avg       0.92      0.71      0.79       500



In [4]:
# !pip install sklearn

import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ✅ Load raw dataset
df = pd.read_csv("train-balanced-sarc.csv", sep="\t", header=None, on_bad_lines="skip")
df.columns = ["label", "author", "subreddit", "score", "parent", "comment_id", "id", "created_utc", "context", "text"]
df = df[["text", "label"]].dropna()
df["label"] = df["label"].astype(int)
df["text"] = df["text"].astype(str)

# ✅ Balance the dataset
min_count = min(df["label"].value_counts()[0], df["label"].value_counts()[1], 50000)  # cap at 20k each

df_sarcastic = df[df["label"] == 1].sample(n=min_count, random_state=42)
df_nonsarcastic = df[df["label"] == 0].sample(n=min_count, random_state=42)
print(len(df_sarcastic), len(df_nonsarcastic))

df_balanced = pd.concat([df_sarcastic, df_nonsarcastic]).sample(frac=1.0, random_state=42).reset_index(drop=True)

# ✅ Convert to Hugging Face dataset
dataset = Dataset.from_pandas(df_balanced)
dataset = dataset.train_test_split(test_size=0.1)

# ✅ Load model and tokenizer
model_name = "helinivan/english-sarcasm-detector"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# ✅ Tokenization
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=64)

tokenized_ds = dataset.map(tokenize, batched=True)
tokenized_ds = tokenized_ds.remove_columns(["text"])
tokenized_ds.set_format("torch")

# ✅ Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Training setup
training_args = TrainingArguments(
    output_dir="./balanced-sarcasm-checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=3e-5,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

# ✅ Save the model
model.save_pretrained("helinivan-finetuned-balanced")
tokenizer.save_pretrained("helinivan-finetuned-balanced")



ModuleNotFoundError: No module named 'pandas'

In [31]:
# ✅ Imports
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ✅ Load and prepare dataset
df = pd.read_csv("train-balanced-sarc.csv", sep="\t", header=None, on_bad_lines="skip")
df.columns = ["label", "author", "subreddit", "score", "parent", "comment_id", "id", "created_utc", "context", "text"]
df = df[["text", "label"]].dropna()
df["label"] = df["label"].astype(int)
df["text"] = df["text"].astype(str)

# ✅ Balance the dataset (up to 50k per class)
min_count = min(df["label"].value_counts()[0], df["label"].value_counts()[1], 50000)
df_sarcastic = df[df["label"] == 1].sample(n=min_count, random_state=42)
df_nonsarcastic = df[df["label"] == 0].sample(n=min_count, random_state=42)
df_balanced = pd.concat([df_sarcastic, df_nonsarcastic]).sample(frac=1.0, random_state=42).reset_index(drop=True)

# ✅ Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df_balanced)
dataset = dataset.train_test_split(test_size=0.1)

# ✅ Load tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Load model from last checkpoint (not from HF Hub)
checkpoint_path = "./cardiff-sarcasm-checkpoints/checkpoint-11250"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

# ✅ Tokenize the dataset
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

tokenized_ds = dataset.map(tokenize, batched=True)
tokenized_ds = tokenized_ds.remove_columns(["text"])
tokenized_ds.set_format("torch")

# ✅ Define metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./cardiff-sarcasm-checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",         # More frequent saving
    save_steps=2000,               # Save every 2000 steps
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

# ✅ Resume training from checkpoint
trainer.train(resume_from_checkpoint=checkpoint_path)

# ✅ Save the final model
model.save_pretrained("cardiff-twitter-roberta-finetuned-sarcasm")
tokenizer.save_pretrained("cardiff-twitter-roberta-finetuned-sarcasm")


Map: 100%|██████████| 10000/10000 [00:00<00:00, 19368.09 examples/s]
/Users/stygianphantom/Library/Python/3.9/lib/python/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/sy/j85r4wzj1b76bxxb72ht66cr0000gn/T/ipykernel_5477/3437571465.py:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
	save_steps: 2000 (from args) != 500 (from trainer_state.json)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
2,0.652000,0.648304,0.626000,0.613461,0.707206,0.657007
3,0.573700,0.681581,0.632200,0.628402,0.670089,0.648576


('cardiff-twitter-roberta-finetuned-sarcasm/tokenizer_config.json',
 'cardiff-twitter-roberta-finetuned-sarcasm/special_tokens_map.json',
 'cardiff-twitter-roberta-finetuned-sarcasm/vocab.json',
 'cardiff-twitter-roberta-finetuned-sarcasm/merges.txt',
 'cardiff-twitter-roberta-finetuned-sarcasm/added_tokens.json',
 'cardiff-twitter-roberta-finetuned-sarcasm/tokenizer.json')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax

# ✅ Load tokenizer and model
model_path = "helinivan-finetuned-balanced"  # or "helinivan/english-sarcasm-detector" if not fine-tuned
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Prediction function
def predict_sarcasm(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=64).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1)
        preds = torch.argmax(probs, dim=1).cpu().tolist()
        confidences = probs[:, 1].cpu().tolist()  # Confidence for "sarcastic" class
    return [{"text": t, "is_sarcastic": bool(p), "confidence": round(c, 3)} for t, p, c in zip(texts, preds, confidences)]

# ✅ Sample sarcastic and non-sarcastic sentences
# test_sentences = [
#     "Oh great, another Monday. Just what I needed.",
#     "Wow, I totally love waiting in line for three hours!",
#     "I absolutely adore it when my code breaks right before the deadline.",
#     "Sure, because sleeping 2 hours a night is super healthy.",
#     "Thanks, I love getting ignored in group projects.",
#     "I just got promoted! Hard work really pays off.",  # Non-sarcastic
#     "The sunset was so beautiful last night.",          # Non-sarcastic
#     "I'm thrilled to pay taxes, best part of the year.", # Sarcastic
# ]

# ✅ Run predictions
results = predict_sarcasm([test_sentence])

# ✅ Print results
for r in results:
    print(f"{'[SARCASM]' if r['is_sarcastic'] else '[GENUINE]'} ({r['confidence']:.2f}): {r['text']}")


[GENUINE] (0.45): Oh great, another Monday. Just what I needed.
[GENUINE] (0.42): Wow, I totally love waiting in line for three hours!
[GENUINE] (0.40): I absolutely adore it when my code breaks right before the deadline.
[SARCASM] (0.53): Sure, because sleeping 2 hours a night is super healthy.
[SARCASM] (0.56): Thanks, I love getting ignored in group projects.
[SARCASM] (0.54): I just got promoted! Hard work really pays off.
[GENUINE] (0.36): The sunset was so beautiful last night.
[SARCASM] (0.64): I'm thrilled to pay taxes, best part of the year.


In [35]:
from transformers import AutoTokenizer, AutoModelWithLMHead

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-sarcasm-twitter")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-sarcasm-twitter")

def detect_sarcasm(text):
    # Encode the input text
    input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')
    # Generate the output
    output = model.generate(input_ids=input_ids, max_length=3)
    # Decode the output
    label = tokenizer.decode(output[0])
    return label.strip()

# List of sentences to test
sentences = [
    "I just love it when my laptop decides to update right before an important meeting. Honestly, I can’t think of a better time.",
    "Of course the printer jammed again. I mean, who doesn’t enjoy fixing tech problems while running late?",
    "Oh sure, because sleeping three hours a night and living off coffee is exactly what they meant by work-life balance.",
    "Nothing beats stepping in a puddle with socks on. Really brightens the whole day.",
    "Wow, the internet is down again during a Zoom presentation. Love how technology always has my back.",
]

# Test each sentence
for sentence in sentences:
    result = detect_sarcasm(sentence)
    print(f"[{result.upper()}]: {sentence}")


[<PAD> NORMAL</S>]: I just love it when my laptop decides to update right before an important meeting. Honestly, I can’t think of a better time.
[<PAD> DERISON]: Of course the printer jammed again. I mean, who doesn’t enjoy fixing tech problems while running late?
[<PAD> DERISON]: Oh sure, because sleeping three hours a night and living off coffee is exactly what they meant by work-life balance.
[<PAD> NORMAL</S>]: Nothing beats stepping in a puddle with socks on. Really brightens the whole day.
[<PAD> NORMAL</S>]: Wow, the internet is down again during a Zoom presentation. Love how technology always has my back.


In [ ]:
# ✅ Install dependencies
# !pip install transformers datasets scikit-learn --quiet

✅ Imports
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ✅ Load and balance the dataset (limit to 50k per class)
df = pd.read_csv("train-balanced-sarc.csv", sep="\t", header=None, on_bad_lines="skip")
df.columns = ["label", "author", "subreddit", "score", "parent", "comment_id", "id", "created_utc", "context", "text"]
df = df[["text", "label"]].dropna()
df["label"] = df["label"].astype(int)
df["text"] = df["text"].astype(str)

min_count = min(df["label"].value_counts()[0], df["label"].value_counts()[1], 50000)
df_sarcastic = df[df["label"] == 1].sample(n=min_count, random_state=42)
df_nonsarcastic = df[df["label"] == 0].sample(n=min_count, random_state=42)
df_balanced = pd.concat([df_sarcastic, df_nonsarcastic]).sample(frac=1.0, random_state=42).reset_index(drop=True)

# ✅ Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df_balanced)
dataset = dataset.train_test_split(test_size=0.1)

# ✅ Tokenizer and Model (roberta-base, 2-class head)
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# ✅ Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

tokenized_ds = dataset.map(tokenize, batched=True)
tokenized_ds = tokenized_ds.remove_columns(["text"])
tokenized_ds.set_format("torch")

# ✅ Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ Training arguments
training_args = TrainingArguments(
    output_dir="./roberta-sarcasm-checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

# ✅ Train!
trainer.train()

# ✅ Save the fine-tuned model
model.save_pretrained("roberta-base-finetuned-sarcasm")
tokenizer.save_pretrained("roberta-base-finetuned-sarcasm")
